Имеется файл с временной статистикой работы асессоров над однотипным заданием. 

Формат файла: login tid Microtasks assigned_ts tclosed_ts 

Пояснение к формату: login — логин асессора; 
                     tid — id оцениваемого задания (task id); 
                     Microtasks – количество микрозаданий в одном задании; 
                     assigned_ts — время резервирования системой задания для асессора; 
                     closed_ts — точное время завершения работы над заданием; 
                     
                     разделитель — табуляция \t. 
                     
Задание может состоять из одного или несколько микрозаданий. Время резервирования задания (assigned_ts) указывает на тот момент, когда система назначила определенного асессора исполнителем этого задания. Этот момент может совпадать с временем начала работы асессора над заданием, а может и не совпадать (асессор может отойти выпить чаю, а потом приступить к заданию, асессор может выполнять предыдущее задание, в то время как за ним зарезервированы новые). 

Предположим, что асессор за 30 секунд своего рабочего времени получает N рублей. Какую оплату вы считаете справедливой для выполнения асессором одного микрозадания из этого файла? Опишите подробно все этапы вашего решения.